In [17]:
from PIL import Image
import skimage.io
import skimage.filters
import skimage.morphology
from skimage.morphology import disk
from skimage.morphology import square
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import os
import re

%matplotlib inline

In [138]:
def get_foci_chloromask_segment(filename, chloro_filename, segment_name, debug):

    #MJR modified 1/27/25
    #this version relies on the chlorophyll fluorescence image to define the cell mask, should remove any
    #possible artifactual correlation between defining the cell boundary and the presence of foci
    
    #MJR modified 5/12/24
    
    #set debug = True when calling this function to see images of each stage
    #of processing
    
    #parameters (may need to be tuned)
    
    cellmaskthreshfactor = 3.5 #how many standard deviations must a 
    #pixel in the median-filtered image to be confident it's inside a cell

    #edited 3/4/25 MJR to allow tunable background subtraction
    backsubfactor = 1.0  #try 0.7
    
    minsigma = 1
    maxsigma = 5
    #essentially smallest and largest features that can be detected
    #using difference-of-gaussians
    
    dog_thresh = 0.002  
    #MJR 1/27/25 changing the threshold
    #dog_thresh_rel = 0.6
    #threshold for minimum intensity needed in peak finding

    
    final_intensity_thresh = 1.2
    #how much does total integrated signal need to rise above background to be counted?
    #in units of std dev of the image
    

    imload = Image.open(filename)
    chloro_imload = Image.open(chloro_filename)
    im = np.array(imload)
    chloro_im = np.array(chloro_imload)
    
    ydim, xdim = im.shape
    if debug:
        print('raw image')
        plt.figure()
        skimage.io.imshow(im)
        plt.show()

    # median filter image (this should remove hot pixels)
    filtersize = 3  #radius of disk used for median filter
    median_im = skimage.filters.median(im, disk(filtersize))
    
    print(type(median_im))

    chloro_median_im = skimage.filters.median(chloro_im, disk(filtersize))

    if debug:
        print(np.max(median_im))
        print(np.min(median_im))
        print(median_im.shape)

    if debug:
        print('median filtered')
        plt.figure()
        skimage.io.imshow(median_im)

    #calculate threshold based on mean and standard deviation ## MJR 1/27/25 not for this anymore: (to get cell mask)
    mu = np.mean(median_im)
    sigma = np.std(median_im)

    chloro_mu = np.mean(chloro_median_im)
    chloro_sigma = np.std(chloro_median_im)

    #MJR 1/27/25 now we use the chlorophyll image
    

    #thresh = mu + cellmaskthreshfactor*sigma
    #changed to constant multiple of mean MJR 5/12/24
    thresh = 1.3*mu
    chloro_thresh = 3.0*chloro_mu  #changed this for the chlorophyll image MJR 1/27/25

    cellmask = chloro_im > chloro_thresh
    


    
    #dilate the cell mask to be sure to get the edges
    cellmask = skimage.morphology.dilation(cellmask, square(3))
    cellmask = skimage.morphology.dilation(cellmask, square(3))
    #cellmask = skimage.morphology.dilation(cellmask, square(3))  #MJR 5/12/24

    threshcomp = lambda t, mask: t*mask
    threshfunc = np.vectorize(threshcomp)
    thresh_im = threshfunc(median_im,cellmask)


    #background subtraction 1/29/25 MJR
    #this should be vectorized...
    tally = 0
    #changed to do median subtraction within cells 
    inmedianlist = []
    outmedianlist = []
    
    for y in range(ydim):
        for x in range(xdim):
            if cellmask[y,x]:
                inmedianlist.append(median_im[y,x])
            else:
                outmedianlist.append(median_im[y,x])
    intense = np.median(inmedianlist) * backsubfactor
    outmedian = np.median(outmedianlist) * backsubfactor
  
    
    backsub = np.copy(median_im)
    outbacksub = np.copy(thresh_im)
    for y in range(ydim):
        for x in range(xdim):
            if median_im[y,x] > intense:
                backsub[y,x] = backsub[y,x] - intense
            else:
                backsub[y,x] = 0
            if outbacksub[y,x] > outmedian:
                outbacksub[y,x] = outbacksub[y,x] - outmedian
            else:
                outbacksub[y,x] = 0
    
    if debug:
        print('cell mask')
        plt.figure()
        skimage.io.imshow(100*cellmask)
    if debug:
        print('thresholded image')
        plt.figure()
        skimage.io.imshow(thresh_im)

    if debug:
        print('backsub image')
        print(type(intense))
        print(intense)
        plt.figure()
        skimage.io.imshow(backsub)

    
    blobs_dog = skimage.feature.blob_dog(skimage.img_as_float(backsub), min_sigma=minsigma, max_sigma=maxsigma, overlap=0.4, threshold=dog_thresh)
   


    #calculate spot intensity by subtracting local median then summing over a patch
    halfwidth = 4
    results = []
    intensity_thresh = final_intensity_thresh*sigma*(halfwidth+1)**2


    if debug:
        fig,axes = plt.subplots()
        axes.imshow(median_im)

    ###3/6/25 DEBUG
    #imload = Image.open(segment_name)
    #segmented = np.array(imload)
    
    for blob in blobs_dog:
        y, x, r = blob
        #safety check that object is not too close to the edge #MJR 5/12/24
        if x > halfwidth and y > halfwidth and x < xdim - halfwidth - 1 and y < ydim - halfwidth - 1:
            #changed to use median-filtered image here MJR 1/27/25
            subimage = im[int(y)-halfwidth:int(y)+halfwidth,int(x)-halfwidth:int(x)+halfwidth]

            
            #subimage = subimage - np.median(subimage)
            #subimage = median_im[int(x)-halfwidth:int(x)+halfwidth,int(y)-halfwidth:int(y)+halfwidth]
            #MJR 5/12/24 use median filtered image to calculate intensity
            subimage = subimage - np.min(subimage)   #MJR 5/12/24 #removed this 1/27/25 becauae now using median-filtered image
            intensity = np.sum(subimage)
            #print('checking on ', blob)
            #if intensity > intensity_thresh:
            #MJR 1/27/25 modify to check if the center is in the cell mask
            if intensity > intensity_thresh and cellmask[round(y),round(x)]:
                result = y, x, intensity
                if debug:
                    c = plt.Circle((x, y), r, color='lime', linewidth=2, fill=False)
                    axes.add_patch(c)
                    print('validated at: ', x, y, r)
                #3/6/25 debugging
                #if segmented[round(y),round(x)] == 0:
                #    print(intensity, 'focus in chlorophyll mask but not segmented')
                #else:
                #    print(intensity, 'focus in cell #', segmented[round(y),round(x)])
                    
                results.append(result)
            else:
                if debug:
                    c = plt.Circle((x, y), r, color='red', linewidth=2, fill=False)
                    axes.add_patch(c)
                    print('invalidated at: ', x, y, r)
                
    if debug:
        print(results)
        plt.show()

    #for summary results, add up area of cell mask
    cell_area = np.sum(cellmask)
    #calculate total intensity of cells
    #total_intensity = np.sum(backsub)
    #MJR 2/4/25 total_intensity should be calculated from the median-filtered masked imaged w/ median background from non-cell area
    total_intensity = np.sum(outbacksub)
    
    foci_intensity = sum([z for x,y,z in results])
    num_foci = len(results)
    intensity_per_area = foci_intensity/cell_area
    total_intensity_per_area = total_intensity/cell_area
    intensity_adj_total = intensity_per_area/total_intensity_per_area

    #MJR 3/5/25
    #repeat analysis on a cell-by-cell basis
    #load segmentation image
    imload = Image.open(segment_name)
    segmented = np.array(imload)
    maxcellID = np.max(segmented)
    seg_analysis = []
    for cellindex in range(1,maxcellID+1):
        if cellindex in segmented:
            #is this index used in the image?
            #mask out the total intensity
            maskout = lambda t, mask: t*mask
            maskfunc = np.vectorize(maskout)
            mask_obs = threshfunc(median_im,segmented==cellindex)
            mask_intensity = np.sum(mask_obs)
            #find all foci in this cell
            tally = 0
            for y,x,z in results:
                if segmented[round(y),round(x)] == cellindex:
                    tally += z
            #if tally > 0:
            #    print('!!!!!!!! foci found in ', cellindex, ' with ', tally/mask_intensity)
            seg_analysis.append((cellindex, tally, mask_intensity, tally / mask_intensity))
    
    #write out to file
    analysis_file = segment_name.replace('.tif','.csv')
    print('writing single cell analysis to ' + analysis_file)
    with open(analysis_file, 'w', newline ='') as file:
        f = csv.writer(file)
        f.writerows(seg_analysis)
                                
    return num_foci, foci_intensity, cell_area, total_intensity, intensity_per_area, total_intensity_per_area, intensity_adj_total

In [65]:
def process_directory_chloro_segment(path):
    #modified MJR 1/27/25
    #we assume that a folder exists containing chlorophyll images that has the same name as "path" except "YFP" is
    #replaced by "Chlorophyll". Further we assume that there is a matched image file in that directory that has the same name
    #as the YFP fluorescence image except with "YFP" replaced by "Chlorophyll"
    chloro_path = path.replace('YFP','Chlorophyll')

    match = re.match(r'^(.*?)_Channel_YFP', path)
    #get prefix containing f.o.v. etc. in filename
    numbers = re.findall(r'\d+', path)
    pairs = [f"{numbers[i]}-{numbers[i+1]}" for i in range(len(numbers)-1)]
    fovmatch =  pairs[-1]
    seg_path = match.group(1) + '_segmentation' + '/'  + fovmatch + '_SegTif/'
    files = os.listdir(path)
    #the key function below is a fix to make sure files with single digit IDs
    #get processed first
    files.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))
    analysis = []
    for name in files:
        print(name)
        #extract frame ID
        IDmatch = re.search(r'\d+', name)
        chloro_name = name.replace('YFP','Chlorophyll')
        segment_name = fovmatch + '_frame_' + str(int(IDmatch.group(0))+1) + '_seg.tif'
        
        filename = path + name
        chloro_filename = chloro_path + chloro_name
        seg_filename = seg_path + segment_name
        print(seg_path)
        print(segment_name)
        num_foci, foci_intensity, cell_area, total_intensity, intensity_per_area, total_intensity_per_area, intensity_adj_total = get_foci_chloromask_segment(filename, chloro_filename, seg_filename, False)
        analysis.append((name, num_foci, foci_intensity, cell_area, total_intensity, intensity_per_area, total_intensity_per_area, intensity_adj_total))
    return analysis

In [139]:
import csv
analysis = process_directory_chloro_segment('/Users/michaelrust/Desktop/working/segtest/2-1_Channel_YFP/')
with open('test.csv', 'w', newline ='') as file:
    f = csv.writer(file)
    f.writerows(analysis)

Cyano_Scan_0_Pos_2_YFP.tif
/Users/michaelrust/Desktop/working/segtest/2-1_segmentation/2-1_SegTif/
2-1_frame_1_seg.tif
<class 'numpy.ndarray'>
40759 focus in cell # 3
41464 focus in cell # 6
40399 focus in cell # 4
35149 focus in cell # 5
40218 focus in cell # 2
39505 focus in cell # 8
!!!!!!!! foci found in  2  with  0.06667805102069577
!!!!!!!! foci found in  3  with  0.034217474342560915
!!!!!!!! foci found in  4  with  0.030021253195410498
!!!!!!!! foci found in  5  with  0.04136563435975592
!!!!!!!! foci found in  6  with  0.03768820698337098
!!!!!!!! foci found in  8  with  0.04520271136172238
writing single cell analysis to /Users/michaelrust/Desktop/working/segtest/2-1_segmentation/2-1_SegTif/2-1_frame_1_seg.csv
Cyano_Scan_1_Pos_2_YFP.tif
/Users/michaelrust/Desktop/working/segtest/2-1_segmentation/2-1_SegTif/
2-1_frame_2_seg.tif
<class 'numpy.ndarray'>
45058 focus in cell # 6
38242 focus in cell # 3
37934 focus in cell # 7
33784 focus in cell # 5
43842 focus in cell # 6
38291 f

FileNotFoundError: [Errno 2] No such file or directory: '/Users/michaelrust/Desktop/working/segtest/2-1_segmentation/2-1_SegTif/2-1_frame_47_seg.tif'

In [131]:
def collate_single_cells(path):
    #for simplicity of use, this function expects the same path name as that for the YFP images
    #then goes to the expected place in the directory structure to get csv files with single cell data
    match = re.match(r'^(.*?)_Channel_YFP', path)
    #get prefix containing f.o.v. etc. in filename
    numbers = re.findall(r'\d+', path)
    pairs = [f"{numbers[i]}-{numbers[i+1]}" for i in range(len(numbers)-1)]
    fovmatch =  pairs[-1]
    seg_path = match.group(1) + '_segmentation' + '/'  + fovmatch + '_SegTif/'

    files = os.listdir(seg_path)
    last_frame = 0
    biggest_cellID = 0
    collated = {}
    for f in files:
        if f.endswith('.csv'):
            #extract frame number from filename
            numbers = re.findall(r'\d+', f)
            frame_no = int(numbers[-1])
            if frame_no > last_frame:
                last_frame = frame_no
            print('processing: ', seg_path + f)
            with open(seg_path + f, "r", newline='') as csvfile:
                data = csv.reader(csvfile,delimiter=',')
                for row in data:
                    #print(row)
                    cellID_s, t, m, ratio_s = row

                    cellID = int(cellID_s)
                    ratio = float(ratio_s)
                    #print(cellID, ratio)
                    if cellID > biggest_cellID:
                        biggest_cellID = cellID
                    if cellID in collated:
                        collated[cellID] = collated[cellID] + [(frame_no, ratio)]
                    else:
                        collated[cellID] = [(frame_no,ratio)]

    print(collated[9])
    #create output table
    results = np.full((last_frame+1,biggest_cellID+1),-1.0)
    for key in collated:
        for frame in collated[key]:
            results[frame[0],key] = frame[1]
            results[frame[0],0] = frame[0] #write frame number to the left-most column
            results[0,key] = key #write cell ID to the top row

    return results


In [140]:
results = collate_single_cells('/Users/michaelrust/Desktop/working/segtest/2-1_Channel_YFP/')
np.savetxt('/Users/michaelrust/Desktop/working/singlecells.csv',results,delimiter=',')

processing:  /Users/michaelrust/Desktop/working/segtest/2-1_segmentation/2-1_SegTif/2-1_frame_40_seg.csv
processing:  /Users/michaelrust/Desktop/working/segtest/2-1_segmentation/2-1_SegTif/2-1_frame_2_seg.csv
processing:  /Users/michaelrust/Desktop/working/segtest/2-1_segmentation/2-1_SegTif/2-1_frame_32_seg.csv
processing:  /Users/michaelrust/Desktop/working/segtest/2-1_segmentation/2-1_SegTif/2-1_frame_22_seg.csv
processing:  /Users/michaelrust/Desktop/working/segtest/2-1_segmentation/2-1_SegTif/2-1_frame_14_seg.csv
processing:  /Users/michaelrust/Desktop/working/segtest/2-1_segmentation/2-1_SegTif/2-1_frame_15_seg.csv
processing:  /Users/michaelrust/Desktop/working/segtest/2-1_segmentation/2-1_SegTif/2-1_frame_41_seg.csv
processing:  /Users/michaelrust/Desktop/working/segtest/2-1_segmentation/2-1_SegTif/2-1_frame_23_seg.csv
processing:  /Users/michaelrust/Desktop/working/segtest/2-1_segmentation/2-1_SegTif/2-1_frame_33_seg.csv
processing:  /Users/michaelrust/Desktop/working/segtest/